In [88]:
import pymongo
import csv
import json

In [32]:
myclient = pymongo.MongoClient("mongodb://localhost:27017/")

In [33]:
mydb = myclient["cs6400"]

In [34]:
print(myclient.list_database_names())

['admin', 'config', 'cs6400', 'local']


In [35]:
print(mydb.list_collection_names())

['result', 'complete']


In [36]:
mycol = mydb['complete']

In [72]:
# Query1: Find number of all crimes in each borough
result = mycol.aggregate([
    # Group the documents and "count" via $sum on the values
    { "$group": {
        "_id": {
            "Borough": "$BORO_NM",
            "Crime_Type" : "$LAW_CAT_CD"
        },
        "#Crimes": { "$sum": 1 }
    }}
])
for x in result:
    print(x)

{'_id': {'Borough': 'QUEENS', 'Crime_Type': 'MISDEMEANOR'}, '#Crimes': 11817}
{'_id': {'Borough': 'MANHATTAN', 'Crime_Type': 'FELONY'}, '#Crimes': 8735}
{'_id': {'Borough': 'MANHATTAN', 'Crime_Type': 'VIOLATION'}, '#Crimes': 4489}
{'_id': {'Borough': 'BROOKLYN', 'Crime_Type': 'FELONY'}, '#Crimes': 10549}
{'_id': {'Borough': 'QUEENS', 'Crime_Type': 'FELONY'}, '#Crimes': 7593}
{'_id': {'Borough': 'STATEN ISLAND', 'Crime_Type': 'MISDEMEANOR'}, '#Crimes': 2161}
{'_id': {'Borough': 'BRONX', 'Crime_Type': 'VIOLATION'}, '#Crimes': 5236}
{'_id': {'Borough': 'BROOKLYN', 'Crime_Type': 'VIOLATION'}, '#Crimes': 7321}
{'_id': {'Borough': 'QUEENS', 'Crime_Type': 'VIOLATION'}, '#Crimes': 5608}
{'_id': {'Borough': 'BROOKLYN', 'Crime_Type': 'MISDEMEANOR'}, '#Crimes': 15671}
{'_id': {'Borough': 'STATEN ISLAND', 'Crime_Type': 'VIOLATION'}, '#Crimes': 1161}
{'_id': {'Borough': 'BRONX', 'Crime_Type': 'FELONY'}, '#Crimes': 6290}
{'_id': {'Borough': 'STATEN ISLAND', 'Crime_Type': 'FELONY'}, '#Crimes': 1100}


In [39]:
# Query3: Rank Boroughs by Women Safety
result = mycol.aggregate([
    #Select rows where victim is Female
    { "$match": { "VIC_SEX": 'F' } },

    # Group the documents and "count" via $sum on the values
    { "$group": {
        "_id": {
            "Borough": "$BORO_NM"
        },
        "#Crimes": { "$sum": 1 }
    }},
    
    #Sort results by #Crimes
    { "$sort": { "#Crimes": 1 } }
])
for x in result:
    print(x)

{'_id': {'Borough': 'STATEN ISLAND'}, '#Crimes': 2300}
{'_id': {'Borough': 'MANHATTAN'}, '#Crimes': 10231}
{'_id': {'Borough': 'BRONX'}, '#Crimes': 11194}
{'_id': {'Borough': 'QUEENS'}, '#Crimes': 11347}
{'_id': {'Borough': 'BROOKLYN'}, '#Crimes': 15233}


In [40]:
# Query 4: Find Age Group Most Targeted for a particular crime (FRAUDS)
result = mycol.aggregate([
    #Select rows Offense is Fraud
    { "$match": { "OFNS_DESC": 'FRAUDS' } },

    # Group the documents and "count" via $sum on the values
    { "$group": {
        "_id": {
            "VICTIM_AGE_GROUP": "$VIC_AGE_GROUP"
        },
        "#Frauds": { "$sum": 1 }
    }},
    
    #Sort results by #Frauds
    { "$sort": { "#Frauds": -1 } }
])
for x in result:
    print(x)

{'_id': {'VICTIM_AGE_GROUP': 'UNKNOWN'}, '#Frauds': 95}
{'_id': {'VICTIM_AGE_GROUP': '45-64'}, '#Frauds': 79}
{'_id': {'VICTIM_AGE_GROUP': '25-44'}, '#Frauds': 72}
{'_id': {'VICTIM_AGE_GROUP': '65+'}, '#Frauds': 22}
{'_id': {'VICTIM_AGE_GROUP': '18-24'}, '#Frauds': 16}
{'_id': {'VICTIM_AGE_GROUP': '<18'}, '#Frauds': 3}


In [66]:
# Query 5: Average number of crimes committed in each borough in each month
result = mycol.aggregate([
    # Group the documents and "count" via $sum on the values
    { "$group": {
        "_id": {
            "Borough": "$BORO_NM",
            "Month" : { '$month' : { '$toDate': "$CMPLNT_FR_DT" }}
        },
        "#Crimes": { "$sum": 1/30 }
    }},
    { "$sort": { "Month": 1, "Borough": 1 } }
])
for x in result:
    print(x)

{'_id': {'Borough': 'QUEENS', 'Month': 12}, '#Crimes': 10.266666666666666}
{'_id': {'Borough': 'STATEN ISLAND', 'Month': 4}, '#Crimes': 18.9}
{'_id': {'Borough': 'QUEENS', 'Month': 2}, '#Crimes': 145.06666666666666}
{'_id': {'Borough': 'STATEN ISLAND', 'Month': 7}, '#Crimes': 0.23333333333333334}
{'_id': {'Borough': 'BRONX', 'Month': 8}, '#Crimes': 0.6333333333333333}
{'_id': {'Borough': 'MANHATTAN', 'Month': 8}, '#Crimes': 1.4}
{'_id': {'Borough': 'QUEENS', 'Month': 11}, '#Crimes': 3.8}
{'_id': {'Borough': 'MANHATTAN', 'Month': 4}, '#Crimes': 109.16666666666667}
{'_id': {'Borough': 'STATEN ISLAND', 'Month': 11}, '#Crimes': 0.5666666666666667}
{'_id': {'Borough': 'BROOKLYN', 'Month': 7}, '#Crimes': 1.4333333333333333}
{'_id': {'Borough': 'BROOKLYN', 'Month': 11}, '#Crimes': 4.466666666666667}
{'_id': {'Borough': 'STATEN ISLAND', 'Month': 12}, '#Crimes': 2.533333333333333}
{'_id': {'Borough': 'BRONX', 'Month': 5}, '#Crimes': 131.63333333333333}
{'_id': {'Borough': 'BROOKLYN', 'Month': 2

In [73]:
# Query 6: Look at crimes against underage women
myquery = { "VIC_SEX": "F", 
          "VIC_AGE_GROUP" : '<18'}

mydoc = mycol.find(myquery)

for x in mydoc:
    print(x)

{'_id': ObjectId('5fc69ff4d841acf8ef95a4d0'), 'CMPLNT_NUM': 352716651, 'RPT_DT': '2020-06-24', 'CMPLNT_FR_DT': '2020-06-24', 'CMPLNT_TO_DT': '2020-06-24', 'CMPLNT_FR_TM': '18:45:00', 'CMPLNT_TO_TM': '18:49:00', 'JURIS_DESC': 'N.Y. POLICE DEPT', 'OFFENSE_CLASS_CODE': 109, 'LAW_CAT_CD': 'FELONY', 'OFNS_DESC': 'GRAND LARCENY', 'BORO_NM': 'BROOKLYN', 'PREM_TYP_DESC': 'DEPARTMENT STORE', 'LOC_OF_OCCUR_DESC': 'FRONT OF', 'SUSP_AGE_GROUP': 'UNKNOWN', 'SUSP_RACE': 'BLACK', 'SUSP_SEX': 'M', 'VIC_AGE_GROUP': '<18', 'VIC_RACE': 'WHITE', 'VIC_SEX': 'F'}
{'_id': ObjectId('5fc69ff4d841acf8ef95a4ea'), 'CMPLNT_NUM': 163815913, 'RPT_DT': '2020-06-25', 'CMPLNT_FR_DT': '2020-06-25', 'CMPLNT_TO_DT': '2020-06-25', 'CMPLNT_FR_TM': '16:30:00', 'CMPLNT_TO_TM': '16:32:00', 'JURIS_DESC': 'N.Y. POLICE DEPT', 'OFFENSE_CLASS_CODE': 578, 'LAW_CAT_CD': 'VIOLATION', 'OFNS_DESC': 'HARRASSMENT 2', 'BORO_NM': 'BRONX', 'PREM_TYP_DESC': 'RESIDENCE-HOUSE', 'LOC_OF_OCCUR_DESC': 'INSIDE', 'SUSP_AGE_GROUP': 'UNKNOWN', 'SUSP_R

{'_id': ObjectId('5fc69ff6d841acf8ef96b24d'), 'CMPLNT_NUM': 115911309, 'RPT_DT': '2020-02-29', 'CMPLNT_FR_DT': '2020-02-29', 'CMPLNT_TO_DT': '2020-02-29', 'CMPLNT_FR_TM': '17:15:00', 'CMPLNT_TO_TM': '17:19:00', 'JURIS_DESC': 'N.Y. POLICE DEPT', 'OFFENSE_CLASS_CODE': 105, 'LAW_CAT_CD': 'FELONY', 'OFNS_DESC': 'ROBBERY', 'BORO_NM': 'BRONX', 'PREM_TYP_DESC': 'RESIDENCE - APT. HOUSE', 'LOC_OF_OCCUR_DESC': 'FRONT OF', 'SUSP_AGE_GROUP': 'UNKNOWN', 'SUSP_RACE': 'WHITE HISPANIC', 'SUSP_SEX': 'M', 'VIC_AGE_GROUP': '<18', 'VIC_RACE': 'BLACK HISPANIC', 'VIC_SEX': 'F'}
{'_id': ObjectId('5fc69ff6d841acf8ef96b262'), 'CMPLNT_NUM': 839792253, 'RPT_DT': '2020-01-03', 'CMPLNT_FR_DT': '2020-03-01', 'CMPLNT_TO_DT': '2020-01-03', 'CMPLNT_FR_TM': '17:00:00', 'CMPLNT_TO_TM': '17:30:00', 'JURIS_DESC': 'N.Y. POLICE DEPT', 'OFFENSE_CLASS_CODE': 105, 'LAW_CAT_CD': 'FELONY', 'OFNS_DESC': 'ROBBERY', 'BORO_NM': 'BRONX', 'PREM_TYP_DESC': 'RESIDENCE - APT. HOUSE', 'LOC_OF_OCCUR_DESC': 'INSIDE', 'SUSP_AGE_GROUP': '<18'

In [77]:
# Query 7: Find all possible type of Felonies
for x in mycol.find({ "LAW_CAT_CD" :'FELONY'}).distinct('OFNS_DESC'):
    print(x)

ARSON
BURGLARY
CHILD ABANDONMENT/NON SUPPORT
CRIMINAL MISCHIEF & RELATED OF
DANGEROUS DRUGS
DANGEROUS WEAPONS
ENDAN WELFARE INCOMP
FELONY ASSAULT
FELONY SEX CRIMES
FORGERY
GAMBLING
GRAND LARCENY
GRAND LARCENY OF MOTOR VEHICLE
HOMICIDE-NEGLIGENT,UNCLASSIFIE
KIDNAPPING & RELATED OFFENSES
MISCELLANEOUS PENAL LAW
NYS LAWS-UNCLASSIFIED FELONY
OTHER STATE LAWS (NON PENAL LA
POSSESSION OF STOLEN PROPERTY
RAPE
ROBBERY
SEX CRIMES
THEFT-FRAUD


In [79]:
# Query 8: Find number of crimes in each borough
result = mycol.aggregate([
    # Group the documents and "count" via $sum on the values
    { "$group": {
        "_id": {
            "Borough": "$BORO_NM"
        },
        "#Crimes": { "$sum": 1 }
    }},
    
    #Sort results by #Crimes
    { "$sort": { "#Crimes": -1 } }
])
for x in result:
    print(x)

{'_id': {'Borough': 'BROOKLYN'}, '#Crimes': 33541}
{'_id': {'Borough': 'MANHATTAN'}, '#Crimes': 28704}
{'_id': {'Borough': 'QUEENS'}, '#Crimes': 25018}
{'_id': {'Borough': 'BRONX'}, '#Crimes': 22932}
{'_id': {'Borough': 'STATEN ISLAND'}, '#Crimes': 4422}


In [89]:
def make_json(csvFilePath, jsonFilePath): 
      
    # create a dictionary 
    data = {} 
      
    # Open a csv reader called DictReader 
    with open(csvFilePath, encoding='utf-8') as csvf: 
        csvReader = csv.DictReader(csvf) 
          
        # Convert each row into a dictionary  
        # and add it to data 
        for rows in csvReader: 
              
            # Assuming a column named 'No' to 
            # be the primary key 
            key = rows['CMPLNT_NUM'] 
            data[key] = rows 
  
    # Open a json writer, and use the json.dumps()  
    # function to dump data 
    with open(jsonFilePath, 'w', encoding='utf-8') as jsonf: 
        jsonf.write(json.dumps(data, indent=4)) 

In [90]:
csvFilePath = r'bogus_complete.csv'
jsonFilePath = r'bogus_complete.json'

In [91]:
make_json(csvFilePath, jsonFilePath)

In [93]:
with open(jsonFilePath) as f:
    file_data = json.load(f)

In [97]:
type(file_data)

dict

In [94]:
mycol.insert_many(file_data)

TypeError: document must be an instance of dict, bson.son.SON, bson.raw_bson.RawBSONDocument, or a type that inherits from collections.MutableMapping

In [98]:
type(mycol)

pymongo.collection.Collection